In [2]:
import pandas as pd
import numpy as np
import csv,sqlite3

In [6]:
trans = pd.read_excel('C:/Users/kenze/Downloads/transactions.xlsx')
calls = pd.read_csv('C:/Users/kenze/Downloads/calls.csv', sep = ';')
trans

,id,date,type,uah,manager,is_packet
0,11000013,2021-01-01,c,4000,Petr,0
1,11000116,2021-01-01,b,5000,Petr,0
2,10347505,2021-01-02,a,1000,Kuat,1
3,10128057,2021-01-03,e,500,Petr,1
4,11000112,2021-01-03,f,9000,Askhat,0
...,...,...,...,...,...,...
994,10291886,2021-10-31,e,500,Petr,1
995,10373467,2021-10-31,f,9000,Petr,1
996,10397524,2021-10-31,f,9000,Ivan,1
997,10791680,2021-10-31,d,10000,Ivan,0


In [65]:
final = trans.merge(calls,left_on='id', right_on='id', suffixes=(False, False))
final

,id,date,type,uah,manager,is_packet,last_call
0,10347505,2021-01-02,a,1000,Kuat,1,23.04.2021
1,10347505,2021-05-03,f,9000,Erbol,1,23.04.2021
2,10128057,2021-01-03,e,500,Petr,1,14.01.2021
3,10128057,2021-09-07,e,500,Aliya,1,14.01.2021
4,10145432,2021-01-04,g,7000,Kuat,1,29.04.2021
...,...,...,...,...,...,...,...
1428,10561570,2021-10-28,e,500,Erbol,1,24.04.2021
1429,10361105,2021-10-22,e,500,Ivan,1,10.06.2021
1430,10812603,2021-10-25,c,4000,Erbol,0,02.01.2021
1431,10179423,2021-10-26,f,9000,Erbol,1,19.05.2021


In [66]:
cnn = sqlite3.connect('sql.db')
final.to_sql('Owner',cnn)
%load_ext sql
%sql sqlite:///sql.db

In [67]:
%%sql
SELECT *
FROM Owner limit 10;

 * sqlite:///sql.db
Done.


index,id,date,type,uah,manager,is_packet,last_call
0,10347505,2021-01-02 00:00:00,a,1000,Kuat,1,23.04.2021
1,10347505,2021-05-03 00:00:00,f,9000,Erbol,1,23.04.2021
2,10128057,2021-01-03 00:00:00,e,500,Petr,1,14.01.2021
3,10128057,2021-09-07 00:00:00,e,500,Aliya,1,14.01.2021
4,10145432,2021-01-04 00:00:00,g,7000,Kuat,1,29.04.2021
5,10145432,2021-06-15 00:00:00,d,10000,Kate,1,29.04.2021
6,10145432,2021-07-06 00:00:00,d,10000,Kuat,1,29.04.2021
7,10154657,2021-01-04 00:00:00,b,5000,Aset,1,29.01.2021
8,10154657,2021-04-13 00:00:00,f,9000,Petr,1,29.01.2021
9,10308638,2021-01-04 00:00:00,f,9000,Askhat,1,26.01.2021


## 1. Из приведенных ниже таблиц составьте запрос, который выберет список уникальных пользователей, которые просматривали веб-страницы на сайте в марте 2021 года, но не купили продукты типа «пакет» в течение этого месяца. Можете использовать MySQL, MS SSQL.

In [68]:
#create the table Visits
visits = {'id':[1,2,3,4,11612],
         'page_id':[5136113,3338852,5984111,1434486,1559772],
         'user_id':[10091236,11081238,10091471,10058112,11085973],
         'server_date':['2021-03-04 23:45:46.000 ','2021-03-05 12:12:14.164','2021-03-07 23:13:24.163' ,'2021-04-12 10:03:38.614','2021-08-09 09:12:51.985'],
          'source':['desktop','ios','ios','android','android']}
visits = pd.DataFrame(data = visits)
#show first 10 elements
visits.head(10)

,id,page_id,user_id,server_date,source
0,1,5136113,10091236,2021-03-04 23:45:46.000,desktop
1,2,3338852,11081238,2021-03-05 12:12:14.164,ios
2,3,5984111,10091471,2021-03-07 23:13:24.163,ios
3,4,1434486,10058112,2021-04-12 10:03:38.614,android
4,11612,1559772,11085973,2021-08-09 09:12:51.985,android


In [69]:
visits['server_date'] = pd.to_datetime(visits['server_date'])
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           5 non-null      int64         
 1   page_id      5 non-null      int64         
 2   user_id      5 non-null      int64         
 3   server_date  5 non-null      datetime64[ns]
 4   source       5 non-null      object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 328.0+ bytes


In [70]:
#create the table Purchases
purchases = {'id':[1,2,3,4,11612],
            'transaction_id':[463,622,646,853,12684],
             'user_id':[10091236,20053412,95021654,51123555,20053411],
             'purchase_date':['2021-03-04 23:45:46.000','2021-03-05 12:12:14.164','2021-03-07 23:13:24.163','2021-04-12 10:03:38.614','2021-08-09 09:12:51.985'],
             'product_type':['vas','packet','packet','vas','packet'],
             'price':[600,1500,1730,600,1200]}

purchases = pd.DataFrame(data = purchases)
#show data
purchases

,id,transaction_id,user_id,purchase_date,product_type,price
0,1,463,10091236,2021-03-04 23:45:46.000,vas,600
1,2,622,20053412,2021-03-05 12:12:14.164,packet,1500
2,3,646,95021654,2021-03-07 23:13:24.163,packet,1730
3,4,853,51123555,2021-04-12 10:03:38.614,vas,600
4,11612,12684,20053411,2021-08-09 09:12:51.985,packet,1200


In [71]:
purchases['purchase_date'] = pd.to_datetime(purchases['purchase_date'])
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              5 non-null      int64         
 1   transaction_id  5 non-null      int64         
 2   user_id         5 non-null      int64         
 3   purchase_date   5 non-null      datetime64[ns]
 4   product_type    5 non-null      object        
 5   price           5 non-null      int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 368.0+ bytes


In [72]:
visits.to_sql('Visits1',cnn)
purchases.to_sql('Purchases1',cnn)

In [73]:
%%sql
select *
from Visits1

 * sqlite:///sql.db
Done.


index,id,page_id,user_id,server_date,source
0,1,5136113,10091236,2021-03-04 23:45:46,desktop
1,2,3338852,11081238,2021-03-05 12:12:14.164000,ios
2,3,5984111,10091471,2021-03-07 23:13:24.163000,ios
3,4,1434486,10058112,2021-04-12 10:03:38.614000,android
4,11612,1559772,11085973,2021-08-09 09:12:51.985000,android


In [74]:
%%sql
select *
from Purchases1

 * sqlite:///sql.db
Done.


index,id,transaction_id,user_id,purchase_date,product_type,price
0,1,463,10091236,2021-03-04 23:45:46,vas,600
1,2,622,20053412,2021-03-05 12:12:14.164000,packet,1500
2,3,646,95021654,2021-03-07 23:13:24.163000,packet,1730
3,4,853,51123555,2021-04-12 10:03:38.614000,vas,600
4,11612,12684,20053411,2021-08-09 09:12:51.985000,packet,1200


In [75]:
%%sql
pragma table_info('Visits1');

 * sqlite:///sql.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,INTEGER,0,None,0
1,id,INTEGER,0,None,0
2,page_id,INTEGER,0,None,0
3,user_id,INTEGER,0,None,0
4,server_date,TIMESTAMP,0,None,0
5,source,TEXT,0,None,0


In [76]:
%%sql
pragma table_info('Purchases1');

 * sqlite:///sql.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,INTEGER,0,None,0
1,id,INTEGER,0,None,0
2,transaction_id,INTEGER,0,None,0
3,user_id,INTEGER,0,None,0
4,purchase_date,TIMESTAMP,0,None,0
5,product_type,TEXT,0,None,0
6,price,INTEGER,0,None,0


In [77]:
%%sql
select *
from Visits1
inner join Purchases1
on Visits1.user_id= Purchases1.user_id;

 * sqlite:///sql.db
Done.


index,id,page_id,user_id,server_date,source,index_1,id_1,transaction_id,user_id_1,purchase_date,product_type,price
0,1,5136113,10091236,2021-03-04 23:45:46,desktop,0,1,463,10091236,2021-03-04 23:45:46,vas,600


## Решение задачи №1

In [89]:
%%sql
select *
from Visits1
where Visits1.user_id = (select distinct(Purchases1.user_id)
                       from Visits1,Purchases1
                       where Visits1.server_date between '2021-03-01 00:00:00' and '2021-03-31 23:59:59'
                       and Purchases1.product_type != 'packet'
                       and Visits1.user_id = Purchases1.user_id);

 * sqlite:///sql.db
Done.


index,id,page_id,user_id,server_date,source
0,1,5136113,10091236,2021-03-04 23:45:46,desktop


## 2. Из приведенных ниже таблиц составьте запрос, чтобы выбрать список уникальных пользователей, 
## которые делали публикации и/или совершали покупки на сайте в течение марта 2021 года.

In [81]:
#create the table Visits
posting = {'user_id':[10091236,10091236,50023697,20014723,11158923],
           'posting_date':['2021-03-04 23:45:46.000','2021-03-05 12:12:14.164','2021-03-07 23:13:24.163','2021-04-12 10:03:38.614','2021-08-09 09:12:51.985'],
           'category_id':[1,1,4,6,6],
           'city_id':[52,21,58,21,21]}
posting = pd.DataFrame(data = posting)
#show first 10 elements
posting['posting_date'] = pd.to_datetime(posting['posting_date'])
posting

,user_id,posting_date,category_id,city_id
0,10091236,2021-03-04 23:45:46.000,1,52
1,10091236,2021-03-05 12:12:14.164,1,21
2,50023697,2021-03-07 23:13:24.163,4,58
3,20014723,2021-04-12 10:03:38.614,6,21
4,11158923,2021-08-09 09:12:51.985,6,21


In [84]:
posting.to_sql('Posting',cnn)

## Решение задачи №2

In [90]:
%%sql 
SELECT * 
FROM Purchases1
WHERE Purchases1.user_id = ( SELECT DISTINCT(Posting.user_id) 
               FROM Posting,Purchases1
               where Posting.posting_date between '2021-03-01 00:00:00' and '2021-03-31 23:59:59'
               and Posting.user_id = Purchases1.user_id ) ;

 * sqlite:///sql.db
Done.


index,id,transaction_id,user_id,purchase_date,product_type,price
0,1,463,10091236,2021-03-04 23:45:46,vas,600


## 3. Из приведенной ниже таблицы составьте запрос, чтобы выбрать вторую последнюю (вторую с конца) покупку каждого юзера

In [88]:
%%sql 
SELECT * 
FROM Purchases1
WHERE Purchases1.purchase_date = ( SELECT MAX(Purchases1.purchase_date) 
               FROM Purchases1
               WHERE Purchases1.purchase_date < ( SELECT MAX(Purchases1.purchase_date) 
                              FROM Purchases1)) ;

 * sqlite:///sql.db
Done.


index,id,transaction_id,user_id,purchase_date,product_type,price
3,4,853,51123555,2021-04-12 10:03:38.614000,vas,600


## 1. Создайте небольшой фрагмент кода, который использует файл Excel, содержащий список менеджеров и их клиентов, в качестве входных данных и создает отдельные файлы Excel <manager’s name.xlsx>, который содержит клиентов для каждого из менеджеров.

In [14]:
manager_name = trans.manager.unique()
manager_name

array(['Petr', 'Kuat', 'Askhat', 'Aset', 'Aliya', 'Kate', 'Ivan',
       'Meruert', 'Erbol', 'Liana'], dtype=object)

In [91]:
kuat = trans[trans['manager'] == 'Kuat']
kuat.to_csv('kuat.csv')
petr = trans[trans['manager'] == 'Petr']
petr.to_csv('petr.csv')
askhat = trans[trans['manager'] == 'Askhat']
askhat.to_csv('askhat.csv')
aset = trans[trans['manager'] == 'Aset']
aset.to_csv('aset.csv')
aliya = trans[trans['manager'] == 'Aliya']
aliya.to_csv('aliya.csv')
kate = trans[trans['manager'] == 'Kate']
kate.to_csv('kate.csv')
ivan = trans[trans['manager'] == 'Ivan']
ivan.to_csv('ivan.csv')
meruert = trans[trans['manager'] == 'Meruert']
meruert.to_csv('meruert.csv')
erbol = trans[trans['manager'] == 'Erbol']
erbol.to_csv('erbol.csv')
liana = trans[trans['manager'] == 'Liana']
liana.to_csv('liana.csv')

## 2. Используя учетные данные ниже для подключения к базе MySQL, напишите скрипт, который подключается к базе данных и запускает некоторый «Select» и записывает результаты в csv файл.

In [57]:
import sqlite3

connection = sqlite3.connect('database.db')
cursor = connection.cursor()
create_table = '''
CREATE TABLE UserBase (
Host VARCHAR(20),
Port VARCHAR(4),
Db VARCHAR(30),
User VARCHAR(20),
Password VARCHAR(30));'''
cursor.execute(create_table)


In [59]:
host = input('Host IP : ')
port = input('Port : ')
db = input('DataBase: ')
user = input('User type: ')
password = input('Password: ')

add_staff = '''INSERT INTO UserBase (Host, Port, Db,User,Password) VALUES (?,?,?,?,?);'''
cursor.execute(add_staff,(host,port,db,user,port))

Host IP :  255.122.2.21
Port :  2554
DataBase:  main
User type:  candidate
Password:  asga


In [63]:
df = pd.read_sql_query("SELECT * FROM UserBase", connection)
df.to_csv('datas.csv')
df

,Host,Port,Db,User,Password
0,255.122.2.21,2554,main,candidate,2554


## 3. Используя данные в excel и csv файлах посчитайте финансовый результат (сумма цен) для каждого менеджера, месяца и продукта. ВАЖНО: Считать только по клиентам, которым звонили и которые являются «пакетными» клиентами. 

In [86]:
consumer = trans.merge(calls)
packet_data = consumer[consumer['is_packet'] == 1]
call = consumer.drop(['id','date','is_packet'], axis = 1)
after_march_calls = call[call['last_call'] > '01.04.2021']
after_march_calls.groupby(['manager','last_call','type']).sum()

uah
manager last_call  type       
Aliya   01.10.2021 a      1000
        02.01.2021 b      5000
        02.03.2021 a      1000
        02.06.2021 a      1000
        02.07.2021 c      4000
...                        ...
Petr    30.07.2021 c      4000
                   d     10000
                   f      9000
        31.01.2021 e      1000
        31.05.2021 b      5000

[1263 rows x 1 columns]